In [2]:
import numpy as np, itertools, networkx as nx, sys, copy,  cv2, os, glob, re, pickle, time as time_lib
from matplotlib import pyplot as plt
from tqdm import tqdm
from collections import defaultdict
def track_time(reset = False):
    if reset:
        track_time.last_time = time_lib.time()
        return '(Initializing time counter)'
    else:
        current_time = time_lib.time()
        time_passed = current_time - track_time.last_time
        track_time.last_time = current_time
        return f'({time_passed:.2f} s)'

In [3]:
mainOutputFolder     = r'.\post_tests'                           # descritive project name e.g [gallium_bubbles, water_bubbles]
mainOutputSubFolders =  ['Field OFF Series 7', 'sccm150-meanFix']   
inputImageFolder     = r'E:\relocated\Downloads\150 sccm' #

intervalStart   = 1                           # start with this ID
numImages       = 1999                         # how many images you want to analyze.
intervalStop    = intervalStart + numImages     # images IDs \elem [intervalStart, intervalStop); start-end will be updated depending on available data.

useMeanWindow   = 0                             # averaging intervals will overlap half widths, read more below
N               = 700                           # averaging window width
rotateImageBy   = cv2.ROTATE_180                # -1= no rotation, cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_90_COUNTERCLOCKWISE, cv2.ROTATE_180 


font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 0.7; thickness = 4;

In [4]:
if 1 == 1:                               # prep image links, get min/max image indexes
    # ================================================================================================
    # ============================= GET PATHS TO IMAGES AND RESORT BY ID =============================
    # 1) get image links
    # 2) extract integers in name [.\img3509, .\img351, .\img3510, ....] -> [3509, 351, 3510, ...]
    # 3) filter by index 
    # 4) sort order by index [.\img3509, .\img351, .\img3510, ...] -> [.\img351, .\img3509, .\img3510, ...]
    imageLinks = glob.glob(inputImageFolder + "**/*.bmp", recursive=True) 
    if len(imageLinks) == 0:
        input("No files inside directory, copy them and press any key to continue...")
        imageLinks = glob.glob(inputImageFolder + "**/*.bmp", recursive=True)                                          # 1)

    extractIntergerFromFileName = lambda x: int(re.findall('\d+', os.path.basename(x))[0])                             # 2)
    imageLinks = list(filter(lambda x: intervalStart <= extractIntergerFromFileName(x) < intervalStop , imageLinks))   # 3)
    imageLinks.sort(key=extractIntergerFromFileName)                                                                   # 4)

    intervalStart   = extractIntergerFromFileName(imageLinks[0])        # update start index, so its captures in subfolder name.
    intervalStop    = extractIntergerFromFileName(imageLinks[-1])       # update end index 


In [5]:
"""
========================================================================================================//
======================================== BUILD PROJECT FOLDER HIERARCHY ================================//
--------------------------------------------------------------------------------------------------------//
------------------------------------ CREATE MAIN PROJECT, SUBPROJECT FOLDERS ---------------------------//
"""
if not os.path.exists(mainOutputFolder): os.mkdir(mainOutputFolder)  
mainOutputSubFolders.append(f"{intervalStart:05}-{intervalStop:05}")       # sub-project folder hierarhy e.g [exp setup, parameter, subset of data]

for folderName in mainOutputSubFolders:     
    mainOutputFolder = os.path.join(mainOutputFolder, folderName)               
    if not os.path.exists(mainOutputFolder): os.mkdir(mainOutputFolder)

# -------------------------------- CREATE VARIOUS OUTPUT FOLDERS -------------------------

imageFolder        = os.path.join(mainOutputFolder, 'images'    )
stagesFolder       = os.path.join(mainOutputFolder, 'stages'    )
dataArchiveFolder  = os.path.join(mainOutputFolder, 'archives'  )
graphsFolder       = os.path.join(mainOutputFolder, 'graphs'    )

[os.mkdir(folder) for folder in (imageFolder, stagesFolder, dataArchiveFolder, graphsFolder) if not os.path.exists(folder)]

imageFolder_pre_run = os.path.join(imageFolder, 'prerun')
if not os.path.exists(imageFolder_pre_run): os.mkdir(imageFolder_pre_run)

imageFolder_output = os.path.join(imageFolder, 'output')
if not os.path.exists(imageFolder_output): os.mkdir(imageFolder_output)


In [6]:
path_modues = r'.\modules'      # os.path.join(mainOutputFolder,'modules')
sys.path.append(path_modues)    

# NOTE: IF YOU USE MODULES, DONT FORGET EMPTY "__init__.py" FILE INSIDE MODULES FOLDER
# NOTE: path_modules_init = os.path.join(path_modues, "__init__.py")
# NOTE: if not os.path.exists(path_modules_init):  with open(path_modules_init, "w") as init_file: init_file.write("")

#--------------------------- IMPORT CUSTOM FUNCITONS -------------------------------------
from cropGUI import cropGUI

from graphs_brects import (overlappingRotatedRectangles)

from image_processing import (convertGray2RGB, undistort)

from bubble_params  import (centroid_area_cmomzz, centroid_area)

from graphs_general import (graph_extract_paths, find_paths_from_to_multi, graph_check_paths, get_connected_components,
                            comb_product_to_graph_edges, for_graph_plots,  extract_clusters_from_edges,
                            set_custom_node_parameters, G2_set_parameters, get_event_types_from_segment_graph)

from graphs_general import (G, G2, key_nodes, keys_segments, G2_t_start, G2_t_end, G2_n_from, G2_n_to, G2_edge_dist, G_time, G_area, G_centroid, G_owner, G_owner_set)

from interpolation import (interpolate_trajectory, extrapolate_find_k_s, interpolateMiddle1D_2, decide_k_s)

from misc import (cyclicColor, timeHMS, modBR, rect2contour, combs_different_lengths, sort_len_diff_f,
                  disperse_nodes_to_times, disperse_composite_nodes_into_solo_nodes, find_key_by_value, CircularBuffer, CircularBufferReverse, 
                  split_into_bins, lr_reindex_masters, dfs_pred, dfs_succ, old_conn_2_new, lr_evel_perm_interp_data, lr_weighted_sols, 
                  save_connections_two_ways, save_connections_merges, save_connections_splits, itertools_product_length, conflicts_stage_1, 
                  conflicts_stage_2, conflicts_stage_3, edge_crit_func, two_crit_many_branches, find_final_master_all,
                  zp_process, f121_disperse_stray_nodes, f121_interpolate_holes, f121_calc_permutations, f121_precompute_params, f121_get_evolutions)


In [8]:
cropMaskName = "-".join(mainOutputSubFolders[:2])+'-crop'
cropMaskPath = os.path.join(os.path.join(*mainOutputFolder.split(os.sep)[:-1]), f"{cropMaskName}.png")
cropMaskMissing = True if not os.path.exists(cropMaskPath) else False

graphsPath          =   os.path.join(dataArchiveFolder  ,  "graphs.pickle"    )
segmentsPath        =   os.path.join(dataArchiveFolder  ,  "segments.pickle"  )
contoursHulls       =   os.path.join(dataArchiveFolder  ,  "contorus.pickle"  )
mergeSplitEvents    =   os.path.join(dataArchiveFolder  ,  "ms-events.pickle" )

                     
meanImagePath       =   os.path.join(dataArchiveFolder  ,  "mean.npz"         )
meanImagePathArr    =   os.path.join(dataArchiveFolder  ,  "meanArr.npz"      )
                     
archivePath         =   os.path.join(stagesFolder       ,  "croppedImageArr.npz"        )
binarizedArrPath    =   os.path.join(stagesFolder       ,  "binarizedImageArr.npz"      )
post_binary_data    =   os.path.join(stagesFolder       ,  "intermediate_data.pickle"   ) 

print(track_time(reset = True))

(Initializing time counter)


In [9]:
if not os.path.exists(archivePath):
    """
    ===================================================================================================
    ======== CROP USING A MASK (DRAW RED RECTANGLE ON EXPORTED SAMPLE IN MANUAL MASK FOLDER) ==========
    IF MASK IS MISSING YOU CAN DRAW IT USING GUI
    """
    if cropMaskMissing: # search for a mask at cropMaskPath (project -> setup -> parameter)
        print(f"\nNo crop mask in {mainOutputFolder} folder!, creating mask : {cropMaskName}.png")
        mapXY           = (np.load('./mapx.npy'), np.load('./mapy.npy'))
        cv2.imwrite(cropMaskPath, convertGray2RGB(undistort(cv2.imread(imageLinks[0],0), mapXY)))

        p1,p2           = cropGUI(cropMaskPath)
        cropMask        = cv2.imread(cropMaskPath,1)

        cv2.rectangle(cropMask, p1, p2,[0,0,255],-1)
        cv2.imwrite(  cropMaskPath,cropMask)
    else:
        cropMask = cv2.imread(cropMaskPath,1)
    # ---------------------------- ISOLATE RED RECTANGLE BASED ON ITS HUE ------------------------------
    cropMask = cv2.cvtColor(cropMask, cv2.COLOR_BGR2HSV)

    lower_red = np.array([(0,50,50), (170,50,50)])
    upper_red = np.array([(10,255,255), (180,255,255)])

    manualMask = cv2.inRange(cropMask, lower_red[0], upper_red[0])
    manualMask += cv2.inRange(cropMask, lower_red[1], upper_red[1])

    # --------------------- EXTRACT MASK CONTOUR-> BOUNDING RECTANGLE (USED FOR CROP) ------------------
    contours = cv2.findContours(manualMask,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    [X, Y, W, H] = cv2.boundingRect(contours[0])
    """
    ===================================================================================================
    =========================== CROP/UNSISTORT AND STORE DATA INTO ARCHIVES ===========================
    ---------------------------------------------------------------------------------------------------
    """
    print(f"\n{timeHMS()}: Processing and saving archive data on drive... {track_time()}\n")
    
    if rotateImageBy % 2 == 0 and rotateImageBy != -1: W,H = H,W     


15-38-19: Processing and saving archive data on drive... (4.49 s)



In [10]:
dataArchive = np.zeros((len(imageLinks),H,W),np.uint8)                  # predefine storage

mapXY       = (np.load('./mapx.npy'), np.load('./mapy.npy'))            # fish-eye correction map

for i,imageLink in tqdm(enumerate(imageLinks), total=len(imageLinks)):
    image = cv2.remap(cv2.imread(imageLink,0), mapXY[0], mapXY[1],cv2.INTER_LINEAR)[Y:Y+H, X:X+W]
    #image = undistort(cv2.imread(imageLink,0), mapXY)[Y:Y+H, X:X+W]
    if rotateImageBy != -1:
        dataArchive[i]    = cv2.rotate(image, rotateImageBy)
    else:
        dataArchive[i]    = image

  9%|▉         | 185/1999 [00:21<03:33,  8.51it/s]


KeyboardInterrupt: 

In [2]:
from multiprocessing import Pool
import worker
if __name__ ==  '__main__': 
 num_processors = 3
 p=Pool(processes = num_processors)
 output = p.map(worker.worker,[i for i in range(0,3)])
 print(output)

[0, 1, 4]


In [3]:
from multiprocessing import Pool
import worker

def a(x):
    return worker.worker(x,0)

if __name__ ==  '__main__': 
    num_processors = 3
    p=Pool(processes = num_processors)
    output = p.map(a,[i for i in range(0,3)])
    print(output)

In [ ]:
dataArchive = np.zeros((len(imageLinks),H,W),np.uint8)                  # predefine storage
from multiprocessing import Pool
import worker

def a(x):
    return worker.worker(x,0)

if __name__ ==  '__main__': 
    num_processors = 3
    p=Pool(processes = num_processors)
    output = p.map(a,[i for i in range(0,3)])
    print(output)
mapXY       = (np.load('./mapx.npy'), np.load('./mapy.npy'))            # fish-eye correction map

for i,imageLink in tqdm(enumerate(imageLinks), total=len(imageLinks)):
    image = cv2.remap(cv2.imread(imageLink,0), mapXY[0], mapXY[1],cv2.INTER_LINEAR)[Y:Y+H, X:X+W]
    #image = undistort(cv2.imread(imageLink,0), mapXY)[Y:Y+H, X:X+W]
    if rotateImageBy != -1:
        dataArchive[i]    = cv2.rotate(image, rotateImageBy)
    else:
        dataArchive[i]    = image

In [8]:
import numpy as np
a = np.array([  [   [1,2],
                    [3,4]   ],

                [   [5,6],
                    [7,8]   ]  ])
b = np.rot90(a, 1, (1,2))
print(b)

[[[2 4]
  [1 3]]

 [[6 8]
  [5 7]]]


In [26]:
import numpy as np

a = [1, 2, 3, 4, 5]
b1 = np.mean(a)
step = 4
num_part = int(np.ceil(len(a) / step))
partition = [a[i:i + step] for i in range(0, num_part * step, step)]

weighted_mean = 0.0
total_weight = 0

for part in partition:
    part_mean = np.mean(part)
    part_weight = len(part)
    
    weighted_mean = (weighted_mean * total_weight + part_mean * part_weight) / (total_weight + part_weight)
    
    total_weight += part_weight

print("Final Weighted Mean:", weighted_mean)
print("Actual Mean:", b1)


Final Weighted Mean: 3.0
Actual Mean: 3.0


In [46]:
import numpy as np
a = np.arange(0,255,10)
thld = 50
delta = (255-thld)
print(a)
a += delta
a = np.clip(a,0,255)
print(a)
a = np.where(a >= 255, a, 0)

print(a)

[  0  10  20  30  40  50  60  70  80  90 100 110 120 130 140 150 160 170
 180 190 200 210 220 230 240 250]
[205 215 225 235 245 255 255 255 255 255 255 255 255 255 255 255 255 255
 255 255 255 255 255 255 255 255]
[  0   0   0   0   0 255 255 255 255 255 255 255 255 255 255 255 255 255
 255 255 255 255 255 255 255 255]


In [101]:
import cv2
import numpy as np
import torch

kernel = np.array([ [1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1] ], dtype=np.float32)#/(9)
kernel = np.ones((3,3),np.int16)
full_area = np.sum(kernel);print(full_area)

im = np.array([ [1, 1, 1, 0, 0],
                [1, 1, 1, 1, 0],
                [1, 1, 1, 1, 1],
                [1, 1, 1, 1, 1],
                [0, 0, 1, 1, 1]], dtype=np.int16)

im = np.array([ [0, 0, 0, 0, 0],
                [0, 1, 0, 0, 0],
                [0, 1, 1, 0, 0],
                [0, 0, 0, 1, 0],
                [0, 0, 0, 0, 0] ], dtype=np.int16)



mode = 1
if mode == 0:
    print(cv2.erode(im, kernel, borderValue = 0))
else:
    print(cv2.dilate(im, kernel, borderValue = 0))

def morph_erode_dilate(im_tensor, kernel, mode):
    padding = (kernel.shape[-1]//2,)*2#;print(padding)
    torch_result0   = torch.nn.functional.conv2d(im_tensor, kernel, padding=padding)
    #print(torch_result0)
    if mode == 0:
        full_area = torch.sum(kernel)
        torch_result0.add_(-full_area + 1)
        #print(torch_result0)
    return torch_result0.clamp_(0, 1)

im_tensor       = torch.tensor(im).unsqueeze(0).unsqueeze(0) # size:(1, 1, 5, 5)
kernel_tensor   = torch.tensor(kernel).unsqueeze(0).unsqueeze(0) # size: (1, 1, 3, 3)
torch_result    = morph_erode_dilate(im_tensor, kernel_tensor, mode = mode)
print(torch_result)
# if 1 == -1:
#     conv            = torch.nn.Conv2d(1, 1, kernel_size = 5, bias = False, padding = 'same', padding_mode ='zeros')
#     conv.weight     = torch.nn.Parameter(kernel_tensor) 
#     torch_result0   = conv(im_tensor).squeeze().squeeze()
# else:
#     padding = (kernel_tensor.shape[-1]//2,)*2;print(padding)
#     torch_result0   = torch.nn.functional.conv2d(im_tensor, kernel_tensor, padding=padding)
# print(torch_result0)
# torch_result0.add_(-full_area + 1)
# torch_result    = torch.clamp(torch_result0, 0, 1)
# torch_result0.add_(1.0)
# T = torch.nn.Threshold(1, 0, inplace=True)
#print(torch_result)

9
[[1 1 1 0 0]
 [1 1 1 1 0]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 1 1 1]]
tensor([[[[1, 1, 1, 0, 0],
          [1, 1, 1, 1, 0],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [0, 0, 1, 1, 1]]]], dtype=torch.int16)


In [33]:
%%timeit
im_tensor       = torch.tensor(im).unsqueeze(0).unsqueeze(0) # size:(1, 1, 5, 5)
kernel_tensor   = torch.tensor(kernel).unsqueeze(0).unsqueeze(0) # size: (1, 1, 3, 3)
torch_result0   = torch.nn.functional.conv2d(im_tensor, kernel_tensor, padding=(1, 1))

81.3 µs ± 15.1 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [38]:
%%timeit
with torch.no_grad():
    im_tensor       = torch.tensor(im).unsqueeze(0).unsqueeze(0) # size:(1, 1, 5, 5)
    kernel_tensor   = torch.tensor(kernel).unsqueeze(0).unsqueeze(0) # size: (1, 1, 3, 3)
    conv            = torch.nn.Conv2d(1, 1, kernel_size = 5, bias = False, padding = 'same', padding_mode ='zeros')
    conv.weight     = torch.nn.Parameter(kernel_tensor) 
    torch_result0   = conv(im_tensor).squeeze().squeeze()

204 µs ± 7.21 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [105]:
N = 1000
imgs0 =  np.stack([im] * N, axis=0);print(imgs0.shape)
kernel = kernel
imgs2 = torch.from_numpy(imgs0).unsqueeze(1);print(imgs2.shape)
kernel_tensor   = torch.tensor(kernel).unsqueeze(0).unsqueeze(0)

(1000, 5, 5)
torch.Size([1000, 1, 5, 5])


In [133]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
imgs2 = imgs2.to(device).to(torch.float)
kernel_tensor = kernel_tensor.to(device).to(torch.float)
print(imgs2.element_size() * imgs2.numel()/ 1024.0,' Kb')

97.65625  Kb


In [134]:
%%timeit
torch_result    = morph_erode_dilate(imgs2, kernel_tensor, mode = 1)

42.8 µs ± 2.26 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [109]:
%%timeit
#dilated_images = np.zeros_like(imgs0)
for i in range(N):
    imgs0[i] = cv2.dilate(imgs0[i], kernel, borderValue = 0)


4.96 ms ± 347 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [124]:
torch.cuda.empty_cache()

In [116]:
import torch, numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import torch.nn as nn

N, H, W = 600, 800, 1200  # Adjust as needed
#N, H, W = 1, 4, 4  # Adjust as needed
torch.manual_seed(44)
# Generate a random tensor with values between 0.0 and 1.0
random_tensor = torch.rand(N, H, W)

# Scale and shift the values to the desired range [-20.0, 255.0]
batch = random_tensor * 275.0 - 30.0
batch.to(device)
#print(batch)

nn_threshold = nn.Threshold(254, 0, inplace=True).to(device)
threshold = 50.0
nn_threshold1 = nn.Threshold(threshold, 0, inplace=True).to(device)

nn_threshold2 = nn.Threshold(0, -255 + 0.001, inplace=True).to(device)




In [109]:
%%timeit
#print(batch)
threshold = 50
batch0 = batch.clone()
#torch.clamp_(batch0, min = 0, max = 255)
delta = (255-threshold + 0.5) # simulate rounding after cast to int by adding 0.5: 0.4->0.9->0 and 0.9->1.4->1
batch0.add_(delta)
batch0 = batch0.to(torch.int16)
#print(batch0)
torch.clamp_(batch0, min = 0, max = 255)
#print(batch0)
nn_threshold(batch0)

#print(batch0)

1.59 s ± 54.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [119]:
#%%timeit
#print(batch)
threshold = 50

batch0 = batch.clone()
batch0 = batch0 > threshold
# torch.clamp_(batch0, min = threshold, max = None)
# #print(batch0)
# batch0.add_(-threshold)
# #print(batch0)
# batch0 = batch0 > 0
print(batch0)

tensor([[[ True,  True,  True,  ...,  True,  True,  True],
         [False, False,  True,  ...,  True, False,  True],
         [ True,  True, False,  ..., False,  True,  True],
         ...,
         [ True,  True, False,  ...,  True, False,  True],
         [ True, False, False,  ...,  True, False, False],
         [ True,  True, False,  ...,  True, False,  True]],

        [[ True, False,  True,  ..., False, False,  True],
         [False,  True, False,  ...,  True,  True, False],
         [ True, False,  True,  ...,  True,  True,  True],
         ...,
         [False, False, False,  ...,  True,  True,  True],
         [ True,  True,  True,  ...,  True,  True,  True],
         [False,  True,  True,  ...,  True,  True, False]],

        [[ True,  True,  True,  ...,  True, False,  True],
         [ True,  True,  True,  ..., False, False,  True],
         [ True,  True,  True,  ...,  True,  True,  True],
         ...,
         [ True,  True, False,  ...,  True,  True, False],
         [

In [77]:
%%timeit
#print(batch)
batch0 = batch.clone()
batch0[batch0 <   threshold]  = 0
batch0[batch0 >=  threshold]  = 255
#print(batch0)

38.5 µs ± 2.31 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [105]:

%%timeit
#print(batch)
batch0 = batch.clone()
nn_threshold1(batch0)

batch0 *= -1.0
batch0 += 0.001

nn_threshold2(batch0)
batch0 -= 0.001
batch0 *= -1.
#print(batch0)

KeyboardInterrupt: 

In [126]:
import cv2
def kernle_circular(width):
    return torch.from_numpy(cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(width,width))).unsqueeze_(0).unsqueeze_(0)
print(kernle_circular(3) + kernle_circular(3))

a = torch.rand(3,1,2,2)
b = torch.rand(1,1,2,2)
print(a + b)

tensor([[[[0, 2, 0],
          [2, 2, 2],
          [0, 2, 0]]]], dtype=torch.uint8)
tensor([[[[1.7854, 1.7363],
          [1.2084, 1.4256]]],


        [[[1.6506, 1.4586],
          [1.3093, 1.3809]]],


        [[[1.2739, 0.9926],
          [1.5014, 1.1683]]]])


In [123]:
import torch

# Size of an individual element in bytes for torch.float and torch.bool
float_element_size = torch.tensor([1.0], dtype=torch.float).element_size()
bool_element_size = torch.tensor([True], dtype=torch.bool).element_size()

print("Size of torch.float element:", float_element_size, "bytes")
print("Size of torch.bool element:", bool_element_size, "bytes")


Size of torch.float element: 4 bytes
Size of torch.bool element: 1 bytes


In [9]:

slice_width = 100

data_length = 551

def gen_slices(data_length, slice_width):
    # input:    data_length, slice_width = 551, 100
    # output:   [(0, 100), (100, 200), (200, 300), (300, 400), (400, 500), (500, 551)] 
    return [(i,min(i+slice_width, data_length)) for i in range(0, data_length, slice_width)]

print(gen_slices(data_length, slice_width)) #[(0, 100), (100, 200), (200, 300), (300, 400), (400, 500), (500, 551)]

[(0, 100), (100, 200), (200, 300), (300, 400), (400, 500), (500, 551)]


In [17]:
def redistribute_vals_bins(values, num_lists):
    # input: values = [0, 1, 2, 3, 4, 5, 6, 7]; num_lists  = 4
    # output : [[0, 4, 8], [1, 5], [2, 6], [3, 7]]
    max_bins = min(len(values),num_lists)
    lists = [[] for _ in range(max_bins)]
    for i, slice_range in enumerate(values):
        list_index = i % max_bins
        lists[list_index].append(slice_range)
    return lists

slices = [0,1,2,3,4,5,6,7,8]
num_lists = 4
redistributed_lists = redistribute_vals_bins(slices, num_lists)
print(redistributed_lists)
# Print the result
for i, lst in enumerate(redistributed_lists):
    print(f"List {i}: {lst}")


[[0, 4, 8], [1, 5], [2, 6], [3, 7]]
List 0: [0, 4, 8]
List 1: [1, 5]
List 2: [2, 6]
List 3: [3, 7]
